In [6]:
from wilcoxon.spiderman import *
from wilcoxon.telegram import *
from wilcoxon import sheets
import re
import itertools
import pandas as pd
from gsheets import Sheets
from wilcoxon import geo
from wilcoxon import utils
from tqdm import tqdm

In [2]:
dir(sheets)

['Sheets',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'codingSheetId',
 'geoSheetId',
 'getCoding',
 'getGeo',
 'getMusic',
 'getVocab',
 'musicSheetId',
 'pd',
 'vocabSheetId']

In [2]:
postcode = sheets.getGeo("Postcode").drop_duplicates(subset = "Postcode").set_index("Postcode").to_dict(orient = "index")

In [3]:
def halalPage(n):

    res = website(f"https://www.halaltag.com/place/{n}")
    
    foot = res.html.find(class_='footer')
    foot.extract()

    title = res.html.find("title").text.split(" - Halal Tag")[0]
    placetype = ", ".join(list(map(lambda x: x.text, res.html(href = re.compile("placetype")))))
    cuisine = ", ".join(list(map(lambda x: x.text, res.html(href = re.compile("cuisine")))))
    food = ", ".join(list(map(lambda x: x.text, res.html(href = re.compile("food")))))
    
    numberLocation = int(res.html("b", text = re.compile("^\d+$"))[0].text)
    numberPages = int((numberLocation - 1) / 50) + 1
    location = []
    
    for pageNumber in range(1, numberPages + 1):
        if pageNumber == 1: locationSite = res
        else: locationSite = website(f"https://www.halaltag.com/place/{n}?page={pageNumber}")
        location.extend(list(map(lambda x: [x.text, str(x.parent.parent.find(text = re.compile("\d{6}"))).strip()], locationSite.html(href = re.compile("location/\d")))))
    
    L = list(itertools.product([title], location, [""], [placetype], [cuisine], [food], [""], [""], [""], [""], [f"https://www.halaltag.com/place/{n}"]))
    resDf = pd.DataFrame(L, columns = ["Name", "Location", "Unit", "Type", "Cuisine", "Food", "Address", "Postal Code", "Latitude", "Longitude", "Link"])
    resDf["Address"] = resDf["Location"].apply(lambda x: x[1])
    resDf["Unit"] = resDf["Address"].str.extract("(#\S*)")
    resDf["Postal Code"] = resDf["Address"].str.extract("(\d{6})")
    resDf["Postal Code"] = pd.to_numeric(resDf["Postal Code"], errors = "coerce").fillna(0)
    resDf["Postal Code"] = resDf["Postal Code"].apply(int)
    resDf["Location"] = resDf["Location"].apply(lambda x: x[0])

    resDf["Latitude"] = resDf["Postal Code"].apply(lambda x: postcode.get(x, {"Lat": None, "Long": None})["Lat"])
    resDf["Longitude"] = resDf["Postal Code"].apply(lambda x: postcode.get(x, {"Lat": None, "Long": None})["Long"])
    
    return resDf


In [12]:
def halalFull():
        
    masterHalal = pd.DataFrame(columns = ["Name", "Location", "Unit", "Type", "Cuisine", "Food", "Address", "Postal Code", "Latitude", "Longitude", "Link"])
    
    for i in tqdm(range(3000)):
        try:
            masterHalal = pd.concat([masterHalal, halalPage(i)])
        except Exception as e:
            print(i, ": ", e)
    
    # for i in Telegram(2000, chatIds = 242141394, botId = "1383233269:AAFj7nsp4fAq9EYQ7cSLpHyTaulQrcK7IVY", timing = 1):
    #     try:
    #         masterHalal = pd.concat([masterHalal, halalPage(i)])
    #     except Exception as e:
    #         print(i, ": ", e)
            
    return masterHalal
    

In [13]:
halalMasterDf = halalFull()

  0%|          | 1/3000 [00:00<42:00,  1.19it/s]

0 :  list index out of range


  0%|          | 5/3000 [00:01<15:22,  3.25it/s]

4 :  list index out of range


  0%|          | 10/3000 [00:04<21:53,  2.28it/s]

...

2995 :  list index out of range
2996 :  list index out of range


100%|█████████▉| 2999/3000 [12:48<00:00,  5.79it/s]

2997 :  list index out of range
2998 :  list index out of range


100%|██████████| 3000/3000 [12:48<00:00,  3.90it/s]

2999 :  list index out of range


In [14]:
halalMasterDf.Latitude.fillna(utils.latlong(halalMasterDf["Postal Code"])[0], inplace=True)
halalMasterDf.Longitude.fillna(utils.latlong(halalMasterDf["Postal Code"])[1], inplace=True)
halalMasterDf["Search"] = halalMasterDf.Name + ", " + halalMasterDf.Type + ", " + halalMasterDf.Cuisine + ", " + halalMasterDf.Food
halalMasterDf = halalMasterDf[["Name","Location","Unit","Search","Address","Postal Code","Latitude","Longitude","Link"]]

0    738099
1    609601
2    556083
3    238872
4     38983
      ...  
0    528833
1    657991
0    199597
0    487048
0     69111
Name: Postal Code, Length: 3294, dtype: object could not be found.
0    738099
1    609601
2    556083
3    238872
4     38983
      ...  
0    528833
1    657991
0    199597
0    487048
0     69111
Name: Postal Code, Length: 3294, dtype: object could not be found.


In [15]:
halalMasterDf.to_clipboard(index=False)

In [16]:
halalMasterDf.name()

AttributeError: 'DataFrame' object has no attribute 'name'